In [ ]:
import pandas as pd
import torch
import numpy as np
from tqdm.notebook import tqdm_notebook as tqdm
from torch.utils.data import DataLoader, TensorDataset

In [ ]:
device = "cpu"  # or "cuda"
dataset_type = "10k"
colab = True  # or True
batch_size = 40

### Setup


In [ ]:
shuf = True
def reset():
    if device == "cuda":
      torch.cuda.empty_cache()
    global X, Y, inp_len, out_len, dataloader

    pathX = f"./datasets/scats/2023/3m/t1_tr1kx.pt"
    pathY = f"./datasets/scats/2023/3m/t1_tr1ky.pt"
    pathEval = f"./datasets/scats/2023/3m/eval.pt"

    if colab:
        from google.colab import drive
        drive.mount('/content/drive', force_remount=False)
        pathX = f"./drive/MyDrive/t2x.pt"
        pathY = f"./drive/MyDrive/t1y.pt"
        pathEval = f"./datasets/scats/2023/3m/eval.pt"

    X = torch.load(pathX)
    Y = torch.load(pathY)
    XE = torch.load(pathEval)

    # float32
    X = X.float()
    Y = Y.float()

    if device == "cuda":
        X = X.cuda()
        Y = Y.cuda()

    inp_len = X.shape[1]*X.shape[2]
    out_len = inp_len

    dataset = TensorDataset(X, Y)
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=shuf)

In [ ]:
def reset_toy():
    torch.cuda.empty_cache()
    global X, Y, inp_len, out_len, dataloader

    def tranform(x):
        return x * 2 + 1
    len = 50
    X = torch.rand(len, 1) * 5
    X.float()
    Y = torch.tensor([tranform(x) for x in X])
    if device == "cuda":
        X = X.cuda()
        Y = Y.cuda()
    Y.unsqueeze_(-1)
    Y.float()
    inp_len = X.shape[1]*X.shape[2]
    out_len = inp_len
    dataset = TensorDataset(X, Y)
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=False)

In [ ]:
reset()

### Model


In [ ]:
def sample_M(m, n, p):
    A = np.random.uniform(0., 1., size=[m, n])
    B = A > p
    C = 1. * B
    return C

def sample_idx(m, n):
    A = np.random.permutation(m)
    idx = A[:n]
    return idx

def sample_Z(m, n, high):
    return np.random.uniform(0., high, size=[m, n])

In [ ]:
miss_rate = 0.1

def mask_for_one():
    mask = sample_M(X.shape[1], 3, miss_rate)
    ones = np.ones(X[0].shape)
    ones[:, -3:] = mask
    return ones


mask = []
for i in range(X.shape[0]):
    mask.append(mask_for_one())

# to tensor
mask = torch.tensor(mask).float()

if device == "cuda":
  mask = mask.cuda()
display(mask.shape)

dataset = TensorDataset(X, mask)
a,b = dataset[0]
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=shuf)

In [ ]:
def discriminator_loss(netG, netD, M, X, H):
    G_sample = netG(X, M)
    Hat_New_X = X * M + G_sample * (1-M)
    D_prob = netD(Hat_New_X, H)
    D_loss = -torch.mean(M * torch.log(D_prob + 1e-8) +
                         (1-M) * torch.log(1. - D_prob + 1e-8))
    return D_loss


def generator_loss(netG, netD, X, M, New_X, H):
    alpha = 2
    G_sample = netG(New_X, M)
    Hat_New_X = New_X * M + G_sample * (1-M)
    D_prob = netD(Hat_New_X, H)
    G_loss1 = -torch.mean((1-M) * torch.log(D_prob + 1e-8))
    MSE_train_loss = torch.mean((M*New_X -M* G_sample)**2)
    G_loss = G_loss1 + alpha * MSE_train_loss
    MSE_test_loss = torch.mean(
        ((1-M) * X - (1-M)*G_sample)**2) / torch.mean(1-M)
    return G_loss, MSE_train_loss, MSE_test_loss

In [ ]:
import torch

def calculate_rmse(actual, predicted):
    assert actual.shape == predicted.shape, "Matrices must have the same shape"
    actual = actual[:,:,-3:]
    predicted = predicted[:,:,-3:]
    squared_differences = (actual - predicted) ** 2
    mean_squared_error = torch.mean(squared_differences)
    rmse = torch.sqrt(mean_squared_error)

    return rmse.item()


In [ ]:
def calculate_mape(actual, predicted, epsilon=3):
    # Ensure the matrices have the same shape
    assert actual.shape == predicted.shape, "Matrices must have the same shape"
    actual = actual[:,:,-3:]
    predicted = predicted[:,:,-3:]

    mask = torch.abs(actual) > epsilon

    actual_filtered = actual[mask]
    predicted_filtered = predicted[mask]

    absolute_percentage_errors = torch.abs((actual_filtered - predicted_filtered) / actual_filtered)

    mape = torch.mean(absolute_percentage_errors) * 100

    return mape.item()

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np

# Generator network

class Generator(nn.Module):
    def __init__(self, input_dim):
        super(Generator, self).__init__()
        layer_dims = [2*input_dim, 1024, 512, input_dim]
        layers = []
        for i in range(len(layer_dims) - 1):
            layers.append(nn.Linear(layer_dims[i], layer_dims[i+1]))
            if i < len(layer_dims) - 2:
                layers.append(nn.ReLU())
        self.layers = layers
        self.model = nn.Sequential(*layers)
        self.init_weight()

    def init_weight(self):
        layers = self.layers
        [torch.nn.init.xavier_normal_(layer.weight) for layer in layers if isinstance(layer, nn.Linear)]

    def forward(self, x, m):
        inp = torch.cat([x, m], dim=1)
        # display(inp.shape)
        out = self.model(inp)
        return out
        # return x*m + out*(1-m)

# Discriminator network


class Discriminator(nn.Module):
    def __init__(self, input_dim):
        # display(input_dim)
        super(Discriminator, self).__init__()
        layer_dims = [2*input_dim, 1024, 512, input_dim]
        layers = []
        for i in range(len(layer_dims) - 1):
            layers.append(nn.Linear(layer_dims[i], layer_dims[i+1]))
            if i < len(layer_dims) - 2:
                layers.append(nn.ReLU())
        self.layers = layers
        self.model = nn.Sequential(*layers)

    def init_weight(self):
        layers = self.layers
        [torch.nn.init.xavier_normal_(layer.weight) for layer in layers if isinstance(layer, nn.Linear)]

    def forward(self, x, m):
        inp = torch.cat([x, m], dim=1)
        return self.model(inp)


G = Generator(inp_len)
D = Discriminator(inp_len)
optim_G = optim.Adam(G.parameters(), lr=2e-4)
optim_D = optim.Adam(D.parameters(), lr=2e-4)
data_iter = iter(dataloader)

if device == "cuda":
    G = G.cuda()
    D = D.cuda()

# criterion = nn.BCELoss()

mb_size = batch_size
hr = 0.5


def gradient_penalty(D, xr, xf):
    """

    :param D:
    :param xr: [b, 2]
    :param xf: [b, 2]
    :return:
    """
    # [b, 1]
    t = torch.rand(batch_size, 1).cuda()
    # [b, 1] => [b, 2]  broadcasting so t is the same for x1 and x2
    t = t.expand_as(xr)
    # interpolation
    mid = t * xr + (1 - t) * xf
    # set it to require grad info
    mid.requires_grad_()

    pred = D(mid)
    grads = torch.autograd.grad(outputs=pred, inputs=mid,
                                grad_outputs=torch.ones_like(pred),
                                create_graph=True, retain_graph=True, only_inputs=True)[0]

    gp = torch.pow(grads.norm(2, dim=1) - 1, 2).mean()

    return gp


# Training loop
epochs = 1000
for epoch in range(epochs):
    cnt = 0
    sum = 0
    for di,(Xi, Msk) in enumerate(dataloader):
        # display(Xi.shape, Msk.shape)
        for _ in range(1):
            X_mb = Xi
            X_mb = X_mb.view(mb_size, -1)
            Z_mb = sample_Z(mb_size, inp_len, 900)

            M_mb = Msk.view(mb_size, -1)
            if device == "cuda":
              M_mb = torch.tensor(M_mb, device="cuda").float()
              X_mb = torch.tensor(X_mb, device="cuda").float()
              Z_mb = torch.tensor(Z_mb, device="cuda").float()
            Hnt = sample_M(mb_size, inp_len, 1-hr)
            if device == "cuda":
              Hnt = torch.tensor(Hnt, device="cuda").float()
            H_mb = M_mb * Hnt
            X_noisy = M_mb * X_mb + (1-M_mb) * Z_mb

            if device == "cuda":
                X_mb = torch.tensor(X_mb, device="cuda").float()
                M_mb = torch.tensor(M_mb, device="cuda").float()
                H_mb = torch.tensor(H_mb, device="cuda").float()
                X_noisy = torch.tensor(X_noisy, device="cuda").float()
                Z_mb = torch.tensor(Z_mb, device="cuda").float()
            else:
                X_mb = torch.tensor(X_mb).float()
                M_mb = torch.tensor(M_mb).float()
                H_mb = torch.tensor(H_mb).float()
                X_noisy = torch.tensor(X_noisy).float()
                Z_mb = torch.tensor(Z_mb).float()


            optim_D.zero_grad()
            Dl = discriminator_loss(G, D, M_mb, X_noisy, H_mb)
            Dl.backward()
            optim_D.step()

        # train G
        optim_G.zero_grad()
        Gl, _,_ = generator_loss(G, D, X_mb, M_mb, X_noisy, H_mb)
        # print(f"loss: {Gl.item()}" )
        Gl.backward()
        optim_G.step()

        out = G(X_mb, M_mb)
        out = out.reshape(mb_size, X.shape[1], X.shape[2])

        print(f"Itr {epoch}/{di} RMSE: {calculate_rmse(out, Xi)} MAPE: {calculate_mape(out, Xi)}")

In [ ]:
# idx = 1
# sample = X[idx].flatten().unsqueeze(dim = 0)
# mmm = mask[idx].flatten().unsqueeze(dim = 0)
# res = G(sample, mmm)
# res = res.squeeze().view(X.shape[1],X.shape[2])
# display(res[:,-3:])
# display(X[idx][:,-3:])

In [ ]:
# # for entire data MAPE
# def eval():
#     G.eval()
#     sz = X.shape[0]
#     sz2 = 3
#     mape = 0
#     tot = 0
#     for i in range(sz):
#         res = G.forward(X[i].flatten().detach(), mask[i].flatten().detach()).detach()
#         for node in range(X[i].shape[0]):
#             for j in range(sz2):
#                 if (abs(X[i][node][j]-1e-6) < 1e-4):
#                     continue
#                 mape += abs(res[node][j]-Y[i][node][j])/(Y[i][node][j]+1e-18)
#                 tot += 1
#     return mape/tot


# eval()*100

In [ ]:
def eval():
  G.eval()
  Z = sample_Z(X.shape[0], inp_len, 900)
  Z = torch.tensor(Z, device=device).float()
  X_eval = X.view(X.shape[0], X.shape[1]*X.shape[2])
  M_eval = mask.view(X.shape[0], X.shape[1]*X.shape[2])
  X_noisy = M_eval*X_eval + (1-M_eval)*Z
  out = G(X_noisy, M_eval)
  out = out.reshape(X.shape[0], X.shape[1], X.shape[2])
  mape = calculate_mape(out,X)
  rmse = calculate_rmse(out,X)
  print(f"mape: {mape} rmse: {rmse}")

eval()